In [1]:
#### Notebook to munge LCD v1 data
#### Raw data downloaded from: https://www.ncei.noaa.gov/products/land-based-station/local-climatological-data
#### Written by: Jennifer.E.Kay@colorado.edu
#### Last update: October 28, 2024 (to include data through September 2024)
#### October 7, 2024 (to separate output files so that they can be uploaded to github)

In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

from metpy.units import units

from datetime import datetime

import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.gridspec import GridSpec
import timeit
import scipy.stats as stats
import glob

In [3]:
foo = pd.read_csv("WBAN03017_1974-1983_3731795.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
foo1 = pd.read_csv("WBAN03017_1984-1993_3731796.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
foo2 = pd.read_csv("WBAN03017_1994-2003_3731797.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
foo3 = pd.read_csv("WBAN03017_2004-2013_3731800.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
foo4 = pd.read_csv("WBAN03017_2014-2023_3731803.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
foo5 = pd.read_csv("WBAN03017_2024_3825800.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
df1 = pd.concat([foo,foo1,foo2,foo3,foo4,foo5])

df1

,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,1974-11-13T02:00:00,12,31,NaN,45.0,NaN,310.0,NaN,15.0
1,1974-11-13T05:00:00,15,30,NaN,54.0,NaN,280.0,NaN,25.0
2,1974-11-13T08:00:00,17,29,NaN,61.0,NaN,280.0,NaN,22.0
3,1974-11-13T11:00:00,20,29,NaN,69.0,NaN,270.0,NaN,11.0
4,1974-11-13T14:00:00,25,29,NaN,85.0,NaN,270.0,NaN,15.0
...,...,...,...,...,...,...,...,...,...
10923,2024-10-22T12:53:00,26,74,NaN,17,NaN,350,NaN,3
10924,2024-10-22T13:53:00,25,76,NaN,15,24.71,230,NaN,5
10925,2024-10-22T14:53:00,24,76,NaN,14,24.7,300,NaN,7
10926,2024-10-22T15:53:00,24,75,NaN,15,24.7,320,NaN,7


In [4]:
included_cols = ['DATE','HourlyDryBulbTemperature',
                 'HourlyDewPointTemperature','HourlyPrecipitation',
                'HourlyRelativeHumidity',
                'HourlyStationPressure','HourlyWindDirection',
                'HourlyWindGustSpeed','HourlyWindSpeed'] # hourly fields only
data1 = df1.loc[:, df1.columns.isin(list(included_cols))]
#df = pd.concat([data0,data1])
df = data1.copy()
print(df.shape)
df

(387351, 9)


,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,1974-11-13T02:00:00,12,31,NaN,45.0,NaN,310.0,NaN,15.0
1,1974-11-13T05:00:00,15,30,NaN,54.0,NaN,280.0,NaN,25.0
2,1974-11-13T08:00:00,17,29,NaN,61.0,NaN,280.0,NaN,22.0
3,1974-11-13T11:00:00,20,29,NaN,69.0,NaN,270.0,NaN,11.0
4,1974-11-13T14:00:00,25,29,NaN,85.0,NaN,270.0,NaN,15.0
...,...,...,...,...,...,...,...,...,...
10923,2024-10-22T12:53:00,26,74,NaN,17,NaN,350,NaN,3
10924,2024-10-22T13:53:00,25,76,NaN,15,24.71,230,NaN,5
10925,2024-10-22T14:53:00,24,76,NaN,14,24.7,300,NaN,7
10926,2024-10-22T15:53:00,24,75,NaN,15,24.7,320,NaN,7


In [5]:
### Print the Shape of the data frame

print(df.shape)

(387351, 9)


In [6]:
#df.to_csv('72476023066_GrandJunction_2002-2021_hourlydata.csv', index=False)
### Read in your data
#df = pd.read_csv("72476023066_GrandJunction_2002-2021_hourlydata.csv",low_memory=False)
### Preview the values in the data frame by printing it
#print(df)

In [7]:
## make the datetime format is a readable datetime

##create a datetime object from a string
#foo = datetime.strptime('2012-01-01', '%Y-%m-%d')
#foo = datetime.strptime('2012-01-01T00:53:00', '%Y-%m-%dT%H:%M:%S')
#foo = datetime.strptime(df['DATE'][0], '%Y-%m-%dT%H:%M:%S')
#print(foo)

df['datetime'] = pd.to_datetime(df['DATE'], format='%Y-%m-%dT%H:%M:%S')
#df

In [8]:
## change the order of the data frame variables
df = df[['DATE','datetime','HourlyDryBulbTemperature','HourlyDewPointTemperature','HourlyPrecipitation',
                'HourlyRelativeHumidity','HourlyStationPressure','HourlyWindDirection','HourlyWindSpeed','HourlyWindGustSpeed']]
df

,DATE,datetime,HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyWindDirection,HourlyWindSpeed,HourlyWindGustSpeed
0,1974-11-13T02:00:00,1974-11-13 02:00:00,31,12,NaN,45.0,NaN,310.0,15.0,NaN
1,1974-11-13T05:00:00,1974-11-13 05:00:00,30,15,NaN,54.0,NaN,280.0,25.0,NaN
2,1974-11-13T08:00:00,1974-11-13 08:00:00,29,17,NaN,61.0,NaN,280.0,22.0,NaN
3,1974-11-13T11:00:00,1974-11-13 11:00:00,29,20,NaN,69.0,NaN,270.0,11.0,NaN
4,1974-11-13T14:00:00,1974-11-13 14:00:00,29,25,NaN,85.0,NaN,270.0,15.0,NaN
...,...,...,...,...,...,...,...,...,...,...
10923,2024-10-22T12:53:00,2024-10-22 12:53:00,74,26,NaN,17,NaN,350,3,NaN
10924,2024-10-22T13:53:00,2024-10-22 13:53:00,76,25,NaN,15,24.71,230,5,NaN
10925,2024-10-22T14:53:00,2024-10-22 14:53:00,76,24,NaN,14,24.7,300,7,NaN
10926,2024-10-22T15:53:00,2024-10-22 15:53:00,75,24,NaN,15,24.7,320,7,NaN


In [9]:
## rename the columns, shorter and include units
df.rename(columns={'HourlyDryBulbTemperature': 'drytemp_F', 'HourlyDewPointTemperature': 'dewtemp_F',
                   'HourlyPrecipitation': 'prec_inch', 'HourlyRelativeHumidity': 'RH',
                   'HourlyStationPressure': 'pres_Hg', 'HourlyWindDirection': 'wdir',
                   'HourlyWindSpeed': 'wspd_mph', 'HourlyWindGustSpeed': 'wgust_mph'}, inplace=True)
df.iloc[0]

DATE         1974-11-13T02:00:00
datetime     1974-11-13 02:00:00
drytemp_F                     31
dewtemp_F                     12
prec_inch                    NaN
RH                          45.0
pres_Hg                      NaN
wdir                       310.0
wspd_mph                    15.0
wgust_mph                    NaN
Name: 0, dtype: object

In [10]:
## convert pressure from a string to a float
df["pres_Hg"] = pd.to_numeric(df["pres_Hg"], downcast="float",errors='coerce')
## Deal with the mix of text and numeric values!  Set text values to NaN using coerce!!

## convert units of pressure from inches Mercury to mbars
#print(df['pres_Hg'][1])
#print((df['pres_Hg'][1]* units.inHg).to(units.mbar))
## example converting to mbar from inches of mercury
print((25.53 * units.inHg).to(units.mbar))

df['pres_Hg']=(df.pres_Hg.values * units('inHg')).to('mbar')

df.rename(columns={'pres_Hg': 'pres_mbar'}, inplace=True)


864.5450198790573 millibar


In [11]:
df["drytemp_F"] = pd.to_numeric(df["drytemp_F"], downcast="float",errors='coerce')
df["dewtemp_F"] = pd.to_numeric(df["dewtemp_F"], downcast="float",errors='coerce')
df["prec_inch"] = pd.to_numeric(df["prec_inch"], downcast="float",errors='coerce')
df["RH"] = pd.to_numeric(df["RH"], downcast="float",errors='coerce')
df["wdir"] = pd.to_numeric(df["wdir"], downcast="float",errors='coerce')
df["wspd_mph"] = pd.to_numeric(df["wspd_mph"], downcast="float",errors='coerce')
df["wgust_mph"] = pd.to_numeric(df["wgust_mph"], downcast="float",errors='coerce')
df

,DATE,datetime,drytemp_F,dewtemp_F,prec_inch,RH,pres_mbar,wdir,wspd_mph,wgust_mph
0,1974-11-13T02:00:00,1974-11-13 02:00:00,31.0,12.0,NaN,45.0,NaN,310.0,15.0,NaN
1,1974-11-13T05:00:00,1974-11-13 05:00:00,30.0,15.0,NaN,54.0,NaN,280.0,25.0,NaN
2,1974-11-13T08:00:00,1974-11-13 08:00:00,29.0,17.0,NaN,61.0,NaN,280.0,22.0,NaN
3,1974-11-13T11:00:00,1974-11-13 11:00:00,29.0,20.0,NaN,69.0,NaN,270.0,11.0,NaN
4,1974-11-13T14:00:00,1974-11-13 14:00:00,29.0,25.0,NaN,85.0,NaN,270.0,15.0,NaN
...,...,...,...,...,...,...,...,...,...,...
10923,2024-10-22T12:53:00,2024-10-22 12:53:00,74.0,26.0,NaN,17.0,NaN,350.0,3.0,NaN
10924,2024-10-22T13:53:00,2024-10-22 13:53:00,76.0,25.0,NaN,15.0,836.776611,230.0,5.0,NaN
10925,2024-10-22T14:53:00,2024-10-22 14:53:00,76.0,24.0,NaN,14.0,836.438049,300.0,7.0,NaN
10926,2024-10-22T15:53:00,2024-10-22 15:53:00,75.0,24.0,NaN,15.0,836.438049,320.0,7.0,NaN


In [12]:
### deal with missing data

## copy the data array into a new array without missing values called (data_nomissing)
df_nomissing=df.copy()
df_nomissing
print(df_nomissing.shape)

option_missing='option2'

##### OPTION #1 = replace the NaN data with the mean value over the entire dataset.  Simple!
if option_missing=='option1':
    df_nomissing['drytemp_F']=df_nomissing['drytemp_F'].fillna(df['drytemp_F'].mean())
    df_nomissing['dewtemp_F']=df_nomissing['dewtemp_F'].fillna(df['dewtemp_F'].mean())
    df_nomissing['prec_inch']=df_nomissing['prec_inch'].fillna(0)
    df_nomissing['RH']=df_nomissing['RH'].fillna(df['RH'].mean())
    df_nomissing['pres_mbar']=df_nomissing['pres_mbar'].fillna(df['pres_mbar'].mean())
    df_nomissing['wdir']=df_nomissing['wdir'].fillna(df['wdir'].mean())
    df_nomissing['wspd_mph']=df_nomissing['wspd_mph'].fillna(df['wspd_mph'].mean())
    df_nomissing['wgust_mph']=df_nomissing['wgust_mph'].fillna(df['wgust_mph'].mean())
    
##### OPTION #2 = ffill: propagate last valid observation forward to next valid observation,
#### then backfill / bfill: use next valid observation to fill gap.
if option_missing=='option2':
     df_nomissing=df_nomissing.fillna(method='bfill')
     df_nomissing=df_nomissing.fillna(method='ffill')
    
df_nomissing


(387351, 10)


/var/folders/5x/myd6c9bx4ml06xc7xyghmj3c0000gp/T/ipykernel_79554/3687420220.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_nomissing=df_nomissing.fillna(method='bfill')
/var/folders/5x/myd6c9bx4ml06xc7xyghmj3c0000gp/T/ipykernel_79554/3687420220.py:25: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_nomissing=df_nomissing.fillna(method='ffill')


,DATE,datetime,drytemp_F,dewtemp_F,prec_inch,RH,pres_mbar,wdir,wspd_mph,wgust_mph
0,1974-11-13T02:00:00,1974-11-13 02:00:00,31.0,12.0,0.0,45.0,1006.773376,310.0,15.0,34.0
1,1974-11-13T05:00:00,1974-11-13 05:00:00,30.0,15.0,0.0,54.0,1006.773376,280.0,25.0,34.0
2,1974-11-13T08:00:00,1974-11-13 08:00:00,29.0,17.0,0.0,61.0,1006.773376,280.0,22.0,34.0
3,1974-11-13T11:00:00,1974-11-13 11:00:00,29.0,20.0,0.0,69.0,1006.773376,270.0,11.0,34.0
4,1974-11-13T14:00:00,1974-11-13 14:00:00,29.0,25.0,0.0,85.0,1006.773376,270.0,15.0,34.0
...,...,...,...,...,...,...,...,...,...,...
10923,2024-10-22T12:53:00,2024-10-22 12:53:00,74.0,26.0,0.0,17.0,836.776611,350.0,3.0,32.0
10924,2024-10-22T13:53:00,2024-10-22 13:53:00,76.0,25.0,0.0,15.0,836.776611,230.0,5.0,32.0
10925,2024-10-22T14:53:00,2024-10-22 14:53:00,76.0,24.0,0.0,14.0,836.438049,300.0,7.0,32.0
10926,2024-10-22T15:53:00,2024-10-22 15:53:00,75.0,24.0,0.0,15.0,836.438049,320.0,7.0,32.0


In [13]:
df_nomissing.to_csv('WBAN03017_1974-2024_hourlydata_munged.csv', index=False)

In [14]:
df.to_csv('WBAN03017_1974-2024_hourlydata_munged_wthmissing.csv', index=False)

In [15]:
df = pd.read_csv("WBAN03017_1974-2024_hourlydata_munged_wthmissing.csv")

In [16]:
df1 = df.loc[(df['datetime'] >= '1995-07-01')
                     & (df['datetime'] < '2015-06-30')]
df1

df1.to_csv('WBAN03017_1995-2015_hourlydata_munged_wthmissing.csv', index=False)

In [18]:
df2 = df.loc[(df['datetime'] >= '2015-07-01')
                     & (df['datetime'] < '2024-10-01')]
df2

df2.to_csv('WBAN03017_2015-2024_hourlydata_munged_wthmissing.csv', index=False)